# MIMIC-III query API tutorial

This notebook shows examples of how to use the cyclops.query API on [MIMIC-III v1.4](https://physionet.org/content/mimiciii/1.4/).

Each query is limit to 100 rows (for quick results).

* First, setup the MIMIC-III database according to the instructions in [mimic-code](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iii/buildmimic/postgres).
* The database is assumed to be hosted using postgres. Update the config parameters such as username and password, passed to `MIMICIIIQuerier` accordingly.

## Imports and instantiate `MIMICIIIQuerier`

In [1]:
"""MIMICIII query API tutorial."""

import cyclops.query.ops as qo
from cyclops.query import MIMICIIIQuerier


querier = MIMICIIIQuerier(
    dbms="postgresql",
    port=5432,
    host="localhost",
    database="mimiciii",
    user="postgres",
    password="pwd",
)
# List all custom table methods.
querier.list_custom_tables()

2023-08-28 13:23:41,419 INFO cyclops.query.orm - Database setup, ready to run queries!


['chartevents', 'diagnoses', 'labevents']

## Example 1. Get all male patients with a mortality outcome.

In [2]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("expire_flag", 1),
        qo.ConditionEquals("gender", "M"),
    ],
)
patients = querier.mimiciii.patients(ops=ops).run(limit=100)
print(f"{len(patients)} rows extracted!")

2023-08-28 13:23:46,246 INFO cyclops.query.orm - Query returned successfully!


2023-08-28 13:23:46,247 INFO cyclops.utils.profile - Finished executing function run_query in 0.026690 s


100 rows extracted!


## Example 2. Get all female patient encounters with diagnoses (`gastroenteritis` in ICD-9 long title).

In [3]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("gender", "F"),
    ],
)
diagnoses_ops = qo.Sequential(
    [
        qo.ConditionSubstring("long_title", "gastroenteritis"),
    ],
)
patients = querier.mimiciii.patients(ops=ops)
admissions = querier.mimiciii.admissions(
    join=qo.JoinArgs(join_table=patients.query, on="subject_id"),
)
diagnoses = querier.diagnoses(
    join=qo.JoinArgs(
        join_table=admissions.query,
        on=["subject_id", "hadm_id"],
    ),
    ops=diagnoses_ops,
).run(limit=100)
print(f"{len(diagnoses)} rows extracted!")

2023-08-28 13:23:47,991 INFO cyclops.query.orm - Query returned successfully!


2023-08-28 13:23:47,992 INFO cyclops.utils.profile - Finished executing function run_query in 0.090329 s


100 rows extracted!


## Example 3. Get potassium lab tests for female patient encounters.

In [4]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("gender", "F"),
    ],
)
patients = querier.mimiciii.patients(ops=ops)
lab_ops = qo.Sequential(
    [
        qo.ConditionEquals("label", "potassium"),
    ],
)
labs = querier.labevents(ops=lab_ops).run(limit=100)
print(f"{len(labs)} rows extracted!")

2023-08-28 13:23:48,044 INFO cyclops.query.orm - Query returned successfully!


2023-08-28 13:23:48,045 INFO cyclops.utils.profile - Finished executing function run_query in 0.023201 s


100 rows extracted!


## Example 4. Get AaDO2 carevue chart events for male patients that have a `valuenum` of less than 20.

In [5]:
ops = qo.Sequential(
    [
        qo.ConditionEquals("gender", "M"),
    ],
)
chartevents_ops = qo.Sequential(
    [
        qo.ConditionEquals("dbsource", "carevue"),
        qo.ConditionEquals("label", "AaDO2"),
        qo.ConditionLessThan("valuenum", 20),
    ],
)
patients = querier.mimiciii.patients(ops=ops)
chart_events = querier.chartevents(ops=chartevents_ops).run(limit=100)
print(f"{len(chart_events)} rows extracted!")

2023-08-28 13:25:01,651 INFO cyclops.query.orm - Query returned successfully!


2023-08-28 13:25:01,653 INFO cyclops.utils.profile - Finished executing function run_query in 73.577140 s


13 rows extracted!
